In [193]:
import pandas as pd
import requests
from tqdm import tqdm

In [194]:
data_url = './make_file/(경사각)자동차_도로_고도차이(경사각)_포함_데이터.csv'

In [195]:
data = pd.read_csv(data_url, encoding="EUC-KR")
data.head(1)

,Unnamed: 0,도로명,도로 종류,시작점_위도,시작점_경도,종료점_위도,종료점_경도,중앙점_위도,중앙점_경도,시작점_고도,...,경사각,도로_길이,터널 여부,교량 여부,서비스 도로 여부,도로 ID,속도 제한,차선 수,일방통행 여부,교차로 여부
0,0,"['광장앞지하차도', '광장로']",3차로,37.545509,127.097772,37.546747,127.102031,37.546128,127.099902,39.830399,...,2.487682,430.542185,yes,NaN,NaN,"[175320385, 175320419, 175320429]",NaN,NaN,NaN,NaN


In [196]:
try :
    data = data.drop(columns="Unnamed: 0")
except :
    print('컬럼 없음')

In [197]:
data["도로 ID"]

0       [175320385, 175320419, 175320429]
1                               516647748
2                               516647752
3                                37401581
4                               516723466
                      ...                
6227                            381409375
6228                           1369902935
6229                           1369902935
6230                           1369902935
6231                           1369902936
Name: 도로 ID, Length: 6232, dtype: object

In [198]:
data.columns

Index(['도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '시작점_고도', '종료점_고도', '고도의_차이', '경사각', '도로_길이', '터널 여부',
       '교량 여부', '서비스 도로 여부', '도로 ID', '속도 제한', '차선 수', '일방통행 여부', '교차로 여부'],
      dtype='object')

## 행정안정부 API

In [224]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
open_api_key_file['사이트'].value_counts()

get_gov_api_key = open_api_key_file[open_api_key_file['사이트'] == '행정안전부'].values[0][1]  

In [230]:
# 변환할 지번 주소 입력
# old_address = "서울특별시 종로구 세종로 1-1"

def old_address_to_road_address(old_address) : 
    url = f"https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey={get_gov_api_key}&keyword={old_address}&resultType=json"
    
    # API 요청
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        # print(data)
        if "results" in data and "juso" in data["results"] and data["results"]["juso"]:
            road_addresses = [juso['roadAddr'] for juso in data['results']['juso']]
            return road_addresses[0]
        else:
            return None
    else:
        return None

In [231]:
# print(old_address_to_road_address("서울시 구의동 604"))

## KAKAO

In [232]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
open_api_key_file['사이트'].value_counts()

get_api_key = open_api_key_file[open_api_key_file['사이트'] == 'kakao_api_key_REST'].values[0][1]  

In [233]:
# KAKAO API KEY 값 
KAKAO_API_KEY = get_api_key  

In [234]:
# 서울시 광진구 위도, 경도

def kakao_map_api(lat, lon) :
    latitude = lat
    longitude = lon
    
    # 카카오 맵 API URL
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    
    # API 요청 헤더
    headers = {
        "Authorization": f"KakaoAK {KAKAO_API_KEY}"
    }
    
    # API 호출
    response = requests.get(url, headers=headers)
    data = response.json()
    # print(data)
    region_3depth_name = data['documents'][0]['address']['region_3depth_name']
    temp_address_name = data['documents'][0]['address']['address_name']
    address_name = data['documents'][0]['road_address'] 
    
    if address_name is None :
        temp = old_address_to_road_address(temp_address_name)
        if temp is None :
            address_name = temp_address_name
            is_road_address = 0 #False
        else : 
            address_name = temp
            is_road_address = 1
    else :
        is_road_address = 1 #True
         
    if isinstance(address_name, dict):
        # print(address_name)
        address_name = address_name.get("address_name", "")  # 기본값을 빈 문자열로 설정
    
    # 문자열에서 특정 단어 제거
    address_name = address_name.replace("서울특별시 ", "")
    address_name = address_name.replace("서울 ", "")
    address_name = address_name.replace("광진구 ", "")

     
    return region_3depth_name, address_name, is_road_address

In [235]:
lat_list = data["중앙점_위도"].to_list()
lon_list = data["중앙점_경도"].to_list()

print(len(lat_list))
print(len(lon_list))

6232
6232


In [236]:
dong_list = []
road_address_list = []
is_road_address_list = []

for i in tqdm(range(len(lat_list))) :
# for i in tqdm(range(60)) :
    lat = lat_list[i]
    lon = lon_list[i]
    dong, road_address, is_road_address = kakao_map_api(lat, lon)
    
    # print(dong, road_address, is_road_address)
    
    dong_list.append(dong)
    road_address_list.append(road_address)
    is_road_address_list.append(is_road_address)

100%|███████████████████████████████████████| 6232/6232 [23:33<00:00,  4.41it/s]


In [238]:
# road_address_list

In [239]:
print(len(dong_list))
print(len(road_address_list))
print(len(is_road_address_list))

6232
6232
6232


In [244]:
data['행정동'] = dong_list
data['주소'] = road_address_list
data['도로명_여부'] = is_road_address_list

In [249]:
data["도로명_여부"].value_counts()

도로명_여부
0    4338
1    1894
Name: count, dtype: int64

In [250]:
data["주소"].value_counts()

주소
능동로 120           168
구의동 15-1           64
자양동 740            51
구의동 241-12         50
자양동 719            38
                 ... 
자양동 553-597         1
자양동 756-1           1
뚝섬로58길 44           1
화양동 12-64           1
자양로26길 8 (구의동)      1
Name: count, Length: 1528, dtype: int64

In [253]:
try :
    save_url = './make_file/(최종)자동차_도로_고도차이(경사각)_포함_데이터.csv'
    data.to_csv(save_url, index=False)
    print("성공~")
except OSError as e :
    print(e)

성공~
